In [2]:
import pandas as pd
import pprint
import urllib
import requests
pp = pprint.PrettyPrinter(indent=4)
#from yanytapi import SearchAPI
from collections import defaultdict
import numpy as np
import csv
import matplotlib.pyplot as plt
import os
from PIL import Image
#from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from gensim.parsing.preprocessing import remove_stopwords
from bs4 import BeautifulSoup
#api = SearchAPI("TjGk9kxFO9ScvfSF8AfeqkXjjujBnz6e")
% matplotlib inline

UsageError: Line magic function `%` not found.


In [1]:
def get_df(years=None):
    if not years:
        return pd.concat([pd.read_csv('articles/' + file) for file in os.listdir('articles') if 'articles' in file], ignore_index=True, sort=False)
    else:
        return pd.concat([pd.read_csv('articles/' + csv_name(year)) for year in years], ignore_index=True, sort=False)

def get_sentence_list(years=None, only_weed=False):
    df = get_df(years=years)
    df['text'] = df['text'].map(process_strings)
    df['text'] = df['text'].map(remove_stopwords)
    if only_weed:
        return [remove_waste(p.split()) for p in df['text'] if 'marijuana' in p or 'cannabis' in p]
    else:
        return [remove_waste(p.split()) for p in df['text']]

def get_list(years=None, only_weed=False):
    return [item for sublist in get_sentence_list(years, only_weed) for item in sublist]

def get_sentence(years=None):
    split_it = get_list(years)
    return ' '.join(split_it)

def process(text):
    text = text.replace("\n", "")
    text = ' '.join(text.split())
    return text

def process_strings(s):
    s = s.lower()
    s = s.replace("’","'")
    s = s.replace("'s","")
    bad_chars = ".;:''?!,\[]”“()\""
    for char in bad_chars:
        s = s.replace(char, "")
    if len(s) > 0 and s[0] == '$':
        return '$'
    return s
    
def csv_name(year):
    return 'articles-' + str(year) + '.csv'

def remove_waste(sentence):
    wasted_words = ['—', '&', '-']
    return [word for word in sentence if word not in wasted_words]

def clean_data(df, verbose=True):
    #df['date'] = pd.to_datetime(df['date'], utc=True).dt.date
    df['date'] = df['date'].apply(lambda s : s[:10])
    df.drop([col for col in df.columns if 'Unnamed' in col], axis=1, inplace=True)
    if verbose:
        print("Original shape: ", df.shape)
    df.drop(df.loc[df['text'] == "We’re sorry, we seem to have lost this page, but we don’t want to lose you."].index, inplace=True)
    df.drop(df.loc[df['text'] == "We’re sorry, we seem to have lost this page, but we don’t want to lose you. Report the broken link here."].index, inplace=True)
    df.drop(df.loc[df['text'] == "Go to Home Page »"].index, inplace=True)
    if verbose:
        print("Dropped failed pages: ", df.shape)
    df.drop(df.loc[(df['text'].str.startswith('Compiled by ')) & (df['p#'] == 1.0)].index, inplace=True)
    df.drop(df.loc[(df['text'].str.startswith('By ')) & (df['p#'] == 1.0)].index, inplace=True)
    if verbose:
        print("Dropped byline paragraphs: ", df.shape)
    df['text'] = df['text'].apply(lambda x : x.replace("\n", ""))
    df['text'] = df['text'].apply(lambda x : ' '.join(x.split()))
    df.drop_duplicates(subset=['date', 'title', 'text', 'p#'], inplace=True)
    if verbose:
        print("Dropped duplicates: ", df.shape)
    return df

def get_summary():
    l = list()
    for year in range(1980, 2020):
        df = clean_data(get_df(years=((year,))), verbose=False)
            l.append({'year': year, 'articles': len(df['id'].unique()), 'tokens' : len(' '.join(df['text']))})
    return pd.DataFrame(l)

In [11]:
from vec_utils import process_paragraphs, get_df
df = get_df(range(1980, 2020))
print(len(df['id'].unique()))
ps = process_paragraphs(df)
words = 0
for k in range(len(ps)):
    words += len(ps[k])
words

19487


TypeError: sequence item 0: expected str instance, list found

In [51]:
years = range(2010, 2020)
with open(get_vocab_name(years), "r") as text_file:
    vocab = text_file.readlines()
for i in range(len(vocab)):
    vocab[i] = vocab[i][:-1]
co_matrix = np.loadtxt(get_matrix_name(years))
backward_vocab = {vocab[n] : n for n in range(len(vocab))}
mar = backward_vocab['marijuana']
can = backward_vocab['cannabis']
top_ten = sorted(vocab, key=lambda word : co_matrix[mar][backward_vocab[word]]+co_matrix[can][backward_vocab[word]])[-11:-1]
for word in top_ten:
    print(word, co_matrix[mar][backward_vocab[word]]+co_matrix[can][backward_vocab[word]])

people 985.0
mr 1055.0
states 1082.0
state 1103.0
recreational 1106.0
legal 1116.0
said 1884.0
use 1930.0
medical 2506.0
cannabis 3624.0


In [161]:
articles = api.search('Medical Marijuana, Pro and Con', begin_date='20060426', end_date='20060426')
for article in articles:
    print(article.byline)
    print(get_identifier(article))
    session = requests
    url = article.web_url
    req = session.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    paragraphs = soup.find_all('p')
    count = 0
    for p in paragraphs:
        print(count, p.get_text())
        count += 1
    break

{'original': None, 'person': [], 'organization': None}
Medical Marijuana, Pro and Con2006-04-26
0 Advertisement
1 APRIL 26, 2006

2 To the Editor:
3 It is no surprise that the Food and Drug Administration dismissed the medical value of cannabis. Plants cannot be patented.
4 An agency with such close ties to the pharmaceutical industry has no incentive to recognize the medical value of a plant that people could grow themselves free of charge.
5 Kenneth Michael White  Ontario, Calif., April 21, 2006
6 The writer is the author of "The Beginning of Today: The Marihuana Tax Act of 1937."
7 We are continually improving the quality of our text archives. Please send feedback, error reports,
        and suggestions to archive_feedback@nytimes.com.
8 A version of this letter appears in print on April 26, 2006, on Page A00018 of the National edition with the headline: Medical Marijuana, Pro and Con.  Today's Paper|Subscribe


9 Go to Home Page »


In [89]:
def get_identifier(article):
    return process(article.headline['main']) + str(pd.to_datetime(article.pub_date)).split(' ')[0]
def get_identifier_set(df):
    return set(df['title'] + df['date'])
df = clean_data(get_df(), verbose=False)
check_set = get_identifier_set(df)

In [61]:
articles = api.search("marijuana", begin_date='20011101', end_date='20011231')

for article in articles:
    print(df.loc[(df['title'] == process(article.headline['main'])) & \
                                    (df['date'] ==  article.pub_date[:10]) & \
                                   (df['p#'] == 1.0)])
    break
    print("Actually, already read ", article.headline['main'])
    print(article.headline['main'])
    print(article.pub_date)
    continue
    session = requests
    url = article.web_url
    req = session.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    paragraphs = soup.find_all('p')
    break

date                                                 id   p#  \
60270  2001-12-15  nyt://article/d42d0908-f405-58ce-9844-7bee522b...  1.0   

                                                    text  \
60270  Research on the medical uses of marijuana is s...   

                                                  title  \
60270  After Two-Decade Halt, Marijuana Research Is Set   

                                                     url  
60270  https://www.nytimes.com/2001/12/15/us/after-tw...  


In [62]:
df.loc[(df['title'] == process('New Drug Czar, Old Problem')) & \
                                    ((df['date'] ==  '2001-05-12T05:00:00+0000'[:10]) & \
                                   (df['p#'] == 1.0))]

,date,id,p#,text,title,url


In [94]:
df.loc[(df['p#'] == 1.0) & (df['date'] ==  '2019-02-02T05:00:00+0000'[:10]) & (df['title'] == 'He Committed Murder. Then He Graduated From an Elite Law School. Would You Hire Him as Your Attorney?')]

,date,id,p#,text,title,url
421858,2019-02-02,nyt://article/82d6002c-0edc-56a0-a209-27c56a31...,1.0,"Last September, a group of academics and activ...",He Committed Murder. Then He Graduated From an...,https://www.nytimes.com/2019/02/02/business/br...


In [92]:
df.loc[(df['p#'] == 1.0) & (df['date'] ==  '2019-02-02T05:00:00+0000'[:10])]['title'][421858]

'He Committed Murder. Then He Graduated From an Elite Law School. Would You Hire Him as Your Attorney?'

In [87]:
df['date'].apply(lambda x : x[:10])

0         1985-12-26
1         1985-12-26
2         1985-12-26
3         1985-12-26
4         1985-12-26
5         1985-12-26
6         1985-12-26
7         1985-12-26
8         1985-12-26
9         1985-12-26
10        1985-12-26
11        1985-12-26
12        1985-12-26
13        1985-12-26
14        1985-12-26
15        1985-12-26
16        1985-12-26
17        1985-12-26
18        1985-12-26
19        1985-12-24
20        1985-12-24
21        1985-12-24
22        1985-12-24
23        1985-12-24
24        1985-12-24
25        1985-12-24
26        1985-12-24
27        1985-12-22
28        1985-12-22
29        1985-12-22
             ...    
421901    2019-02-02
421902    2019-02-02
421903    2019-02-02
421904    2019-02-02
421905    2019-02-02
421906    2019-02-02
421907    2019-02-02
421908    2019-02-02
421909    2019-02-02
421910    2019-02-02
421911    2019-02-02
421912    2019-02-02
421913    2019-02-02
421914    2019-02-02
421915    2019-02-02
421916    2019-02-02
421917    201

In [5]:
get_summary()

,year,articles,tokens
0,1980,248,80969
1,1981,369,1983291
2,1982,336,1627674
3,1983,313,1591410
4,1984,332,1856424
5,1985,360,1875560
6,1986,501,2842728
7,1987,430,2413639
8,1988,379,2369749
9,1989,357,2060262
